In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from DeepPurpose.pybiomed_helper import _GetPseudoAAC, CalculateAADipeptideComposition, \
calcPubChemFingerAll, CalculateConjointTriad, GetQuasiSequenceOrder
import torch
from torch.utils import data
from torch.autograd import Variable
try:
	from descriptastorus.descriptors import rdDescriptors, rdNormalizedDescriptors
except:
	raise ImportError("Please install pip install git+https://github.com/bp-kelley/descriptastorus.")
from DeepPurpose.chemutils import get_mol, atom_features, bond_features, MAX_NB, ATOM_FDIM, BOND_FDIM
from subword_nmt.apply_bpe import BPE
import codecs
import pickle
import wget
from zipfile import ZipFile 
import os
import sys

from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import SequentialSampler
from torch import nn 

from tqdm import tqdm
import matplotlib.pyplot as plt
from time import time
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, f1_score, log_loss
from lifelines.utils import concordance_index
from scipy.stats import pearsonr
import pickle 
torch.manual_seed(2)
np.random.seed(3)
import copy
from prettytable import PrettyTable

import os

from DeepPurpose.utils import *
from DeepPurpose.model_helper import Encoder_MultipleLayers, Embeddings        
from DeepPurpose.encoders import *
from DeepPurpose import DTI

In [13]:
data_path = './data//BindingDB_All.tsv'
df = pd.read_csv(data_path, sep = '\t', error_bad_lines=False)
df = df[df['Number of Protein Chains in Target (>1 implies a multichain complex)'] == 1.0]
df = df[df['Ligand SMILES'].notnull()]

b'Skipping line 772572: expected 193 fields, saw 205\nSkipping line 772598: expected 193 fields, saw 205\n'
b'Skipping line 805291: expected 193 fields, saw 205\n'
b'Skipping line 827961: expected 193 fields, saw 265\n'
b'Skipping line 1231688: expected 193 fields, saw 241\n'
b'Skipping line 1345591: expected 193 fields, saw 241\nSkipping line 1345592: expected 193 fields, saw 241\nSkipping line 1345593: expected 193 fields, saw 241\nSkipping line 1345594: expected 193 fields, saw 241\nSkipping line 1345595: expected 193 fields, saw 241\nSkipping line 1345596: expected 193 fields, saw 241\nSkipping line 1345597: expected 193 fields, saw 241\nSkipping line 1345598: expected 193 fields, saw 241\nSkipping line 1345599: expected 193 fields, saw 241\n'
b'Skipping line 1358864: expected 193 fields, saw 205\n'
b'Skipping line 1378087: expected 193 fields, saw 241\nSkipping line 1378088: expected 193 fields, saw 241\nSkipping line 1378089: expected 193 fields, saw 241\nSkipping line 1378090: e

In [20]:
df = df[['BindingDB Reactant_set_id', 'Ligand InChI', 'Ligand SMILES',\
                  'PubChem CID', 'UniProt (SwissProt) Primary ID of Target Chain',\
                  'Target Source Organism According to Curator or DataSource',\
                  'BindingDB Target Chain  Sequence', 'Kd (nM)', 'IC50 (nM)', 'Ki (nM)',\
                  'EC50 (nM)', 'kon (M-1-s-1)', 'koff (s-1)','pH','Temp (C)']]
df.rename(columns={'BindingDB Reactant_set_id':'ID',
                        'Ligand SMILES':'SMILES',
                        'Ligand InChI':'InChI',
                        'PubChem CID':'PubChem_ID',
                        'UniProt (SwissProt) Primary ID of Target Chain':'UniProt_ID',
                        'BindingDB Target Chain  Sequence': 'Target Sequence',
                        'Target Source Organism According to Curator or DataSource': 'Organism',
                        'Kd (nM)':'Kd',
                        'IC50 (nM)':'IC50',
                        'Ki (nM)':'Ki',
                        'EC50 (nM)':'EC50',
                        'kon (M-1-s-1)':'kon',
                        'koff (s-1)':'koff',
                        'Temp (C)':'Temp'}, 
                        inplace=True)

C:\Users\Julia\anaconda3\envs\multiPurpose\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
df.head()

,ID,InChI,SMILES,PubChem_ID,UniProt_ID,Organism,Target Sequence,Kd,IC50,Ki,EC50,kon,koff,pH,Temp
0,1,InChI=1S/C22H24BrFN4O2/c1-28-7-5-14(6-8-28)12-...,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,3081361.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,NaN,NaN,0.24,NaN,NaN,NaN,5.5,37.00 C
1,2,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,5327236.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,NaN,NaN,0.25,NaN,NaN,NaN,5.5,37.00 C
2,3,InChI=1S/C29H34N4O3/c34-27-25(16-21-8-3-1-4-9-...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,5327235.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,NaN,NaN,0.41,NaN,NaN,NaN,5.5,37.00 C
3,4,InChI=1S/C29H40N2O4/c32-18-10-2-1-9-17-30-25(1...,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,5327234.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,NaN,NaN,0.8,NaN,NaN,NaN,5.5,37.00 C
4,5,InChI=1S/C28H38N2O4/c31-17-9-3-8-16-29-24(18-2...,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,3009319.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,NaN,NaN,0.99,NaN,NaN,NaN,5.5,37.00 C


In [38]:
df['Temp'] = df['Temp'].str.rstrip('C')
df.count()

ID                 1733850
InChI              1733282
SMILES             1733850
PubChem_ID         1718479
UniProt_ID         1538086
Organism           1238470
Target Sequence    1733850
Kd                   74761
IC50               1080811
Ki                  417859
EC50                164210
kon                    654
koff                   524
pH                  204919
Temp                191364
dtype: int64

In [36]:
df.to_pickle("./df.pkl")

In [2]:
df = pd.read_pickle("./df.pkl")

In [3]:
idx_str = ['IC50','Temp']
df_want = df
convert_to_log = 0

# have at least uniprot or pubchem ID
df_want = df_want[df_want.PubChem_ID.notnull() | df_want.UniProt_ID.notnull()]
df_want = df_want[df_want.InChI.notnull()]
df_want = df_want[df_want['Temp'].notnull()]
df_want = df_want[df_want['pH'].notnull()]
df_want = df_want[df_want['IC50'].notnull()]

for label in idx_str:
#    print(df_want.size)
    df_want[label] = df_want[label].str.replace('>', '')
    df_want[label] = df_want[label].str.replace('<', '')
    #df_want[label] = df_want[label].astype(float)
#    df_want = df_want[df_want[label] <= 10000000.0]

y = df_want[idx_str]
for label in idx_str:
    if convert_to_log:
            print('Default set to logspace (nM -> p) for easier regression')
            y[label] = convert_y_unit(df_want[label].values, 'nM', 'p') 
    else:
            y[label] = df_want[label].values

y['pH'] = df_want['pH']
X_drugs = df_want.SMILES.values
X_targets = df_want['Target Sequence'].values
y = y.apply(pd.to_numeric, errors='coerce')

C:\Users\Julia\anaconda3\envs\multiPurpose\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Julia\anaconda3\envs\multiPurpose\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
df_want

,ID,InChI,SMILES,PubChem_ID,UniProt_ID,Organism,Target Sequence,Kd,IC50,Ki,EC50,kon,koff,pH,Temp
180,181,InChI=1/C31H51N5O5/c1-20(2)28(32-22(5)37)30(40...,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,65023.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,8.5,NaN,NaN,NaN,NaN,6.0,37.00
181,182,InChI=1/C33H55N5O7/c1-7-44-32(42)35-28(22(3)4)...,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,461984.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,177,NaN,NaN,NaN,NaN,6.0,37.00
183,184,InChI=1/C35H59N5O9/c1-24(2)30(37-34(44)48-19-1...,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,461988.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,164,NaN,NaN,NaN,NaN,6.0,37.00
184,185,InChI=1/C39H67N5O11/c1-28(2)34(41-38(48)54-23-...,COCCOCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc...,461990.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,67,NaN,NaN,NaN,NaN,6.0,37.00
185,186,InChI=1/C38H51N7O7/c1-24(2)34(43-38(51)52-3)37...,COC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](Cc...,461985.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,27,NaN,NaN,NaN,NaN,6.0,37.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062791,50319347,InChI=1S/C26H33N3O2/c1-27(18-15-21-11-5-4-6-12...,CN(CCCCCCn1c(O)c2Cc3ccccc3Cn2c1=O)CCc1ccccc1,91936612.0,Q99720,Homo sapiens,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...,NaN,29.5,NaN,NaN,NaN,NaN,7.5,22.00
1751877,51029634,"InChI=1S/C38H65NO12/c1-15-26-38(10,45-14)31(41...",CCC1OC(=O)C(C)C(O[C@H]2C[C@@](C)(OC)[C@@H](O)C...,44276817.0,NaN,NaN,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN,8.9,NaN,NaN,NaN,NaN,2.5,25.00
1751879,51029639,"InChI=1S/C37H67NO13/c1-14-25-37(10,45)30(41)20...",CCC1OC(=O)C(C)C(O[C@H]2C[C@@](C)(OC)[C@@H](O)C...,10897911.0,NaN,NaN,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN,71,NaN,NaN,NaN,NaN,2.5,25.00
1751880,51029642,"InChI=1/C37H65NO12/c1-14-25-37(10,43)30(40)20(...",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,83954.0,NaN,NaN,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN,224,NaN,NaN,NaN,NaN,2.5,25.00


In [4]:
X_org = df_want['Organism']
y

,IC50,Temp,pH
180,8.5,37.0,6.0
181,177.0,37.0,6.0
183,164.0,37.0,6.0
184,67.0,37.0,6.0
185,27.0,37.0,6.0
...,...,...,...
1062791,29.5,22.0,7.5
1751877,8.9,25.0,2.5
1751879,71.0,25.0,2.5
1751880,224.0,25.0,2.5


In [11]:
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer

imputer = SimpleImputer() #TODO: better imputer
y_i = imputer.fit_transform(y)

In [6]:
y_i = pd.DataFrame(data=y,columns=idx_str+['pH'])
y_i

,IC50,Temp,pH
180,8.5,37.0,6.0
181,177.0,37.0,6.0
183,164.0,37.0,6.0
184,67.0,37.0,6.0
185,27.0,37.0,6.0
...,...,...,...
1062791,29.5,22.0,7.5
1751877,8.9,25.0,2.5
1751879,71.0,25.0,2.5
1751880,224.0,25.0,2.5


In [7]:
df_data = y
df_data['SMILES'] = X_drugs
df_data['Target Sequence'] = X_targets
df_data['Organism'] = X_org

print('in total: ' + str(len(df_data)) + ' drug-target pairs')
df_data

in total: 93309 drug-target pairs


,IC50,Temp,pH,SMILES,Target Sequence,Organism
180,8.5,37.0,6.0,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1
181,177.0,37.0,6.0,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1
183,164.0,37.0,6.0,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1
184,67.0,37.0,6.0,COCCOCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1
185,27.0,37.0,6.0,COC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](Cc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1
...,...,...,...,...,...,...
1062791,29.5,22.0,7.5,CN(CCCCCCn1c(O)c2Cc3ccccc3Cn2c1=O)CCc1ccccc1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...,Homo sapiens
1751877,8.9,25.0,2.5,CCC1OC(=O)C(C)C(O[C@H]2C[C@@](C)(OC)[C@@H](O)C...,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN
1751879,71.0,25.0,2.5,CCC1OC(=O)C(C)C(O[C@H]2C[C@@](C)(OC)[C@@H](O)C...,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN
1751880,224.0,25.0,2.5,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN


In [8]:
df_backup = df_data
df_data = df_data.head(10000)

In [9]:
df_backup.IC50

0          1.264663e+08
1          1.264663e+08
2          1.264663e+08
3          1.264663e+08
4          1.264663e+08
               ...     
1730861    1.264663e+08
1730862    1.264663e+08
1730863    1.264663e+08
1730864    1.264663e+08
1730865    1.264663e+08
Name: IC50, Length: 1730866, dtype: float64

In [67]:
import math
def magnitude(x):
    if x > 0:
        return int(math.floor(math.log10(x)))
    else:
        return 0

magic50 = df_data['IC50'].apply(magnitude)

In [20]:
df_foo = df_data
df_foo['magic50'] = magic50
chaindf = df_foo.groupby(by=['magic50']).count()
chaindf

,IC50,Temp,pH,SMILES,Target Sequence,Organism
magic50,,,,,,
-4,1,1,1,1,1,0
-3,113,113,113,113,113,15
-2,262,262,262,262,262,148
-1,4632,4632,4632,4632,4632,2123
0,17711,17719,17719,17719,17719,10094
1,23355,23355,23355,23355,23355,13384
2,20336,20336,20336,20336,20336,12008
3,12899,12899,12899,12899,12899,7538
4,9646,9646,9646,9646,9646,6161


In [5]:
del df_want
del df

In [8]:
import time

drug_func_list= [smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,smiles2daylight]
#TODO: add calcPubChemFingerAll back in when it's not broken
#TODO: smiles2rdkit2d takes forever and can be added later
#TODO: smiles2mpnnfeature doesn't take super long (around 40 min on desktop) but can be added later
#TODO: same wrt smiles2daylight
column_name = 'SMILES'
start = time.time()

for func in drug_func_list:
    save_column_name = func.__name__
    unique = pd.Series(df_data[column_name].unique()).apply(func)
    unique_dict = dict(zip(df_data[column_name].unique(), unique))
    df_data[save_column_name] = [unique_dict[i] for i in df_data[column_name]]
    end = time.time()
    print(end - start)

rdkit not found this smiles for morgan: CSc1ccc(cc1)C1=C(C=C[N]([O-])=C1)[C@@H]1CCC(F)(F)C[C@H]1C(=O)NCC#N convert to all 0 features
rdkit not found this smiles for morgan: O=C1NC(=O)c2c1c1c3ccccc3n3[Ru](C#[O])[n+]4cccc2c4c13 convert to all 0 features
rdkit not found this smiles for morgan: CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(CC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(CC[N@@]23CC(=O)O[Re]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: CN1C=C[N]2=C1C[N]1(CCc3ccc(SOON)cc3)CC3=[N](C=CN3C)[Re+]21 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(NC(=S)NCCCCCCCC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(NC(=S)NCCOCCOCC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 c

rdkit not found this smiles: CC[N]1(CC)CCN(CCOc2ccc3[nH]c(=O)c4CCCNc4c3c2)CC1 convert to all 0 features
rdkit not found this smiles: COc1cc(C[N](C)(C)C2CCCC2)cc2[nH]c(=O)c3CCCNc3c12 convert to all 0 features
rdkit not found this smiles: CC1=C2C(N(C(=O)C2=[N](N1)C1CC1)c1cc(C)c(=O)n(C)c1)c1ccc(Cl)cc1 convert to all 0 features
rdkit not found this smiles: CC1=N[N](C)=C(C)C1N[S+]([O-])(=O)c1c(Cl)cc(cc1Cl)-c1cccc2CNCCc12 convert to all 0 features
rdkit not found this smiles: CNC(=O)C1=CC(CCCN2CCOCC2)[N](=C1C)c1ccccc1 convert to all 0 features
rdkit not found this smiles: O=C(N1N=C(CC1c1ccccc1)c1cnc2ccccc12)c1ccccc1 convert to all 0 features
rdkit not found this smiles: CCOC(=O)CCc1c(CC(=O)OCC)cnc1\C=N\O convert to all 0 features
rdkit not found this smiles: CC(C)(O)c1nc(ncc1F)N1C[C@H]2CSC(N)=[N]([C@H]2C1)c1cnccn1 convert to all 0 features
rdkit not found this smiles: CC(C)NC(=O)[C@@H]1C[C@H]2C[C@H]2N1C(=O)C[N]1#[N]=C(C(C)=O)c2cc(OCc3ncccn3)ccc12 convert to all 0 features
rdkit not found thi

In [9]:
prot_func_list = [CalculateConjointTriad, protein2emb_encoder,target2quasi]
#TODO: run CalculateAADipeptideComposition and _GetPseudoAAC when time permits
#TODO: GetQuasiSequenceOrder is broken
#prot_func_list = [GetQuasiSequenceOrder]
column_name = 'Target Sequence'
start = time.time()

for func in prot_func_list:
    save_column_name = func.__name__
    AA = pd.Series(df_data[column_name].unique()).apply(func)
    AA_dict = dict(zip(df_data[column_name].unique(), AA))
    df_data[save_column_name] = [AA_dict[i] for i in df_data[column_name]]
    end = time.time()
    print(end - start)

1.0716609954833984
35.09021234512329
Quasi-seq fingerprint not working for smiles: mgsskskpkdpsqrrcsleppdsthhggfpasqtpnktaapdthrtpsrsfgtvatepklfggfntsdtvtspqragalaggvttfvalydyesrtetdlsfkkgerlqivnntegdwwlahslttgqtgyipsnyvapsdsiqaeewyfgkitrreserlllnpenprgtflvresettkgayclsvsdfdnakglnvkhykirkldsggfyitsrtqfsslqqlvayyskhadglchrltnvcptskpqtqglakdaweipreslrlevklgqgcfgevwmgtwngttrvaiktlkpgtmspeaflqeaqvmkklrheklvqlyavvseepiyivteymskgslldflkgemgkylrlpqlvdmaaqiasgmayvermnyvhrdlraanilvgenlvckvadfglarliedneytarqgakfpikwtapeaalygrftiksdvwsfgilltelttkgrvpypgmgngevldrvergyrmpcppecpeslhdlmcqcwrrdpeerptfeylqaqllpacvlevae convert to 0 vectors
Quasi-seq fingerprint not working for smiles: mlpstiqtltlfltsggvllslyvsaslsyllysdillkfspkitaptmtldctnasnvqavnrsatkemtfllpepewtyprlscqgstfqkallisphrfgeargnsapliirepfiacgpkeckhfalthyaaqpggyyngtredrnklrhlisvklgkiptvensifhmaawsgsachdgrewtyigvdgpdsnalikikygeaytdtyhsyannilrtqesacnciggdcylmitdgsasgiskcrflkiregriikeifptgrvehteectcgfasnktiecacrdnnytakrpfvklnvetdtaeirlmctetyld

74.5247552394867


In [12]:
df_data.to_pickle("./df_data_small.pkl")

In [12]:
df_data = pd.read_pickle("./df_data_small3.pkl")

In [3]:
import tables

ImportError: DLL load failed: The specified module could not be found.

In [23]:
import sys
np.set_printoptions(threshold=sys.maxsize)

df_data.to_csv('df_data_ph.csv.gz'
         , sep='|'
         , header=True
         , index=False
         , chunksize=5000
         , compression='gzip'
         , encoding='utf-8')

In [6]:
df_data = pd.read_csv('df_data_ph.csv.gz'
         , sep='|'
         , compression='gzip'
         , encoding='utf-8')

In [72]:
df_data.head()

,IC50,Temp,pH,SMILES,Target Sequence,Organism,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,smiles2daylight,CalculateConjointTriad,protein2emb_encoder,target2quasi
180,8.5,37.0,6.0,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2266, 117, 72, 124, 339, 295, 186, 277, 1880...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000..."
181,177.0,37.0,6.0,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([240, 339, 1416, 295, 186, 277, 1880, 1436, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000..."
183,164.0,37.0,6.0,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2227, 339, 1416, 295, 186, 277, 1880, 1436, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000..."
184,67.0,37.0,6.0,COCCOCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([867, 486, 339, 1416, 295, 186, 277, 1880, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000..."
185,27.0,37.0,6.0,COC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](Cc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 5.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([92, 2346, 199, 179, 1397, 206, 763, 124, 81,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000..."


In [10]:
#turn categorical variables into numerical dummy variables for modeling

cat_list = pd.get_dummies(df_data['Organism'], prefix='var')
df_data=df_data.join(cat_list)

In [74]:
df_data1

,IC50,Temp,pH,SMILES,Target Sequence,Organism,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,smiles2daylight,...,var_Sus scrofa,var_Torpedo marmorata,var_Trichomonas vaginalis G3,var_Trypanosoma brucei,var_Trypanosoma brucei brucei,var_Trypanosoma cruzi,var_Vibrio proteolyticus,var_Xenopus laevis,var_Yersinia pestis,var_[Bacteroides] pectinophilus ATCC 43243
180,8.5,37.0,6.0,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2266, 117, 72, 124, 339, 295, 186, 277, 1880...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
181,177.0,37.0,6.0,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([240, 339, 1416, 295, 186, 277, 1880, 1436, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
183,164.0,37.0,6.0,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2227, 339, 1416, 295, 186, 277, 1880, 1436, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
184,67.0,37.0,6.0,COCCOCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([867, 486, 339, 1416, 295, 186, 277, 1880, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
185,27.0,37.0,6.0,COC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](Cc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 5.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([92, 2346, 199, 179, 1397, 206, 763, 124, 81,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062791,29.5,22.0,7.5,CN(CCCCCCn1c(O)c2Cc3ccccc3Cn2c1=O)CCc1ccccc1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...,Homo sapiens,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([277, 87, 168, 285, 557, 468, 171, 70, 124, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...","[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
1751877,8.9,25.0,2.5,CCC1OC(=O)C(C)C(O[C@H]2C[C@@](C)(OC)[C@@H](O)C...,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([622, 182, 107, 32, 361, 122, 820, 1885, 583,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
1751879,71.0,25.0,2.5,CCC1OC(=O)C(C)C(O[C@H]2C[C@@](C)(OC)[C@@H](O)C...,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([622, 182, 107, 32, 361, 122, 820, 1885, 583,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",...,0,0,0,0,0,0,0,0,0,0
1751880,224.0,25.0,2.5,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,MGSPWNGSDGPEDAREPPWAALPPCDERRCSPFPLGTLVPVTAVCL...,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([1953, 1644, 122, 820, 1885, 186, 1287, 99, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0


In [11]:
discard=['SMILES','Target Sequence','Organism']
#discard=['SMILES','Target Sequence','Organism','trans_drug','trans_protein','pH_raw','Kd_raw','IC50_raw','Ki_raw','EC50_raw','Temp_raw']
df_vars=df_data.columns.values.tolist()
to_keep=[i for i in df_vars if i not in discard]
df_final=df_data[to_keep]
df_final.head()

,IC50,Temp,pH,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,smiles2daylight,CalculateConjointTriad,protein2emb_encoder,target2quasi,...,var_Sus scrofa,var_Torpedo marmorata,var_Trichomonas vaginalis G3,var_Trypanosoma brucei,var_Trypanosoma brucei brucei,var_Trypanosoma cruzi,var_Vibrio proteolyticus,var_Xenopus laevis,var_Yersinia pestis,var_[Bacteroides] pectinophilus ATCC 43243
180,8.5,37.0,6.0,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2266, 117, 72, 124, 339, 295, 186, 277, 1880...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000...",...,0,0,0,0,0,0,0,0,0,0
181,177.0,37.0,6.0,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([240, 339, 1416, 295, 186, 277, 1880, 1436, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000...",...,0,0,0,0,0,0,0,0,0,0
183,164.0,37.0,6.0,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2227, 339, 1416, 295, 186, 277, 1880, 1436, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000...",...,0,0,0,0,0,0,0,0,0,0
184,67.0,37.0,6.0,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([867, 486, 339, 1416, 295, 186, 277, 1880, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000...",...,0,0,0,0,0,0,0,0,0,0
185,27.0,37.0,6.0,"[0.0, 5.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([92, 2346, 199, 179, 1397, 206, 763, 124, 81,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...","[0.000344, 0.000459, 0.000459, 0.000459, 0.000...",...,0,0,0,0,0,0,0,0,0,0


In [12]:
#X = df_final.drop(["IC50","magic50","Kd","Ki","EC50","smiles2mpnnfeature"], axis=1)
X = df_final.drop(["IC50"], axis=1)
#X2 = X

In [53]:
X["smiles2morgan"].iloc[1]

['0.',
 '6.',
 '1.',
 '0.',
 '3.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '1.',
 '0.',
 '0.',
 '0.',
 '6.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '2.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '3.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '1.',
 '5.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '2.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '3.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '4.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',
 '0.',

In [30]:
import collections

In [31]:
def flattener(x):
    if isinstance(x, collections.Iterable):
        return [a for i in x for a in flattener(i)]
    else:
        return [x]

smiles2morgan : <class 'str'>
drug2emb_encoder : <class 'str'>
calcPubChemFingerAll : <class 'str'>
smiles2daylight : <class 'str'>
CalculateConjointTriad : <class 'str'>
protein2emb_encoder : <class 'str'>
target2quasi : <class 'str'>

In [62]:
from array import array

X['drug2emb_encoder'].apply(eval)

NameError: name 'array' is not defined

In [54]:
from ast import literal_eval

for n in range(len(X)):
    #X['CalculateConjointTriad'][n] = X['CalculateConjointTriad'][n].str.strip('[]').str.split()
    #X['smiles2morgan'][n] = X['smiles2morgan'][n].str.strip('[.]').str.split()
    X['drug2emb_encoder'][n] = literal_eval(X['drug2emb_encoder'][n])
    X['calcPubChemFingerAll'][n] = X['calcPubChemFingerAll'][n].str.strip('[]').str.split()
    X['smiles2daylight'][n] = X['smiles2daylight'][n].str.strip('[.]').str.split()
    X['protein2emb_encoder'][n] = literal_eval(X['protein2emb_encoder'][n])
    X['target2quasi'][n] = X['target2quasi'][n].str.strip('[]').str.split()
X.head()

ValueError: malformed node or string: <_ast.Call object at 0x000002342F7CA388>

In [13]:
typelist = []
for i in list(X):
    print(i, ':', type(X[i].iloc[1]))#, ':', X[i].iloc[1])
    if isinstance(X[i].iloc[1],np.ndarray):
        print(len(X[i].iloc[1]))
        typelist.extend([i]*len(X[i].iloc[1]))
    elif isinstance(X[i].iloc[1],tuple):
        for n in X[i].iloc[1]:
            if isinstance(n,np.ndarray):
                print(len(n))
                typelist.extend([i]*len(n))
    elif i[:4] == 'var_':
        typelist.append('organism')
    else:
        typelist.append(i)

Temp : <class 'numpy.float64'>
pH : <class 'numpy.float64'>
smiles2morgan : <class 'numpy.ndarray'>
1024
drug2emb_encoder : <class 'tuple'>
50
50
calcPubChemFingerAll : <class 'numpy.ndarray'>
881
smiles2daylight : <class 'numpy.ndarray'>
2048
CalculateConjointTriad : <class 'numpy.ndarray'>
343
protein2emb_encoder : <class 'tuple'>
545
545
target2quasi : <class 'numpy.ndarray'>
100
var_Abelson murine leukemia virus : <class 'numpy.uint8'>
var_Agaricus bisporus : <class 'numpy.uint8'>
var_Aspergillus fumigatiaffinis : <class 'numpy.uint8'>
var_Avian sarcoma virus : <class 'numpy.uint8'>
var_Bacillus anthracis : <class 'numpy.uint8'>
var_Bacillus cereus (strain ATCC 14579 / DSM 31) : <class 'numpy.uint8'>
var_Bombyx mori : <class 'numpy.uint8'>
var_Borrelia burgdorferi : <class 'numpy.uint8'>
var_Bos taurus : <class 'numpy.uint8'>
var_Caenorhabditis elegans : <class 'numpy.uint8'>
var_Canavalia ensiformis : <class 'numpy.uint8'>
var_Candida albicans : <class 'numpy.uint8'>
var_Canis lup

In [81]:
Z_df = np.empty(shape=[len(X),len(flattener(X.iloc[0]))])
for n in range(len(X)):
    Z_df[n] = flattener(X.iloc[n])

In [84]:
np.savetxt("Z_df.csv.gz", Z_df, delimiter=",")

In [14]:
Z_df=pd.read_csv('Z_df.csv.gz', sep=',',header=None)

In [3]:
vecx = X.iloc[0:100].apply(flattener,axis=1)

NameError: name 'X' is not defined

In [23]:
vec = X.apply(flattener,axis=1)

In [16]:
len(Z_df)

93309

In [15]:
foo2 = Z_df.to_numpy(dtype = object)
#foo2 = np.vstack(Z_df)
foo2


KeyboardInterrupt



In [17]:
# tag each index with equal probabilities to True & False
# note: p = m1 / ( m1 + m2 ) can make more sense depending
# on desired distribution
tag = np.random.binomial(n=1, p=.8, size=len(Z_df)) == 1

# assign True indices to idx1 and False indices to index 2
idx = np.array( range( len(Z_df) ) )
idx1, idx2 = idx[ tag ], idx[ np.logical_not( tag ) ]

# sample from idx1 and idx2
i1, i2 = np.random.choice( idx1, size=10000 ), np.random.choice( idx2, size=10000 )

In [21]:
df_data.head()

,IC50,Temp,pH,SMILES,Target Sequence,Organism,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,smiles2daylight,...,var_Sus scrofa,var_Torpedo marmorata,var_Trichomonas vaginalis G3,var_Trypanosoma brucei,var_Trypanosoma brucei brucei,var_Trypanosoma cruzi,var_Vibrio proteolyticus,var_Xenopus laevis,var_Yersinia pestis,var_[Bacteroides] pectinophilus ATCC 43243
180,8.5,37.0,6.0,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2266, 117, 72, 124, 339, 295, 186, 277, 1880...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
181,177.0,37.0,6.0,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([240, 339, 1416, 295, 186, 277, 1880, 1436, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
183,164.0,37.0,6.0,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([2227, 339, 1416, 295, 186, 277, 1880, 1436, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
184,67.0,37.0,6.0,COCCOCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 6.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([867, 486, 339, 1416, 295, 186, 277, 1880, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0
185,27.0,37.0,6.0,COC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](Cc...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Human immunodeficiency virus 1,"[0.0, 5.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, ...","([92, 2346, 199, 179, 1397, 206, 763, 124, 81,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,0,0,0,0,0,0,0,0,0,0


In [23]:
Z_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5658,5659,5660,5661,5662,5663,5664,5665,5666,5667
0,37.0,6.0,0.0,6.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,37.0,6.0,0.0,6.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,37.0,6.0,0.0,6.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,37.0,6.0,0.0,6.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,37.0,6.0,0.0,5.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
X_train = Z_df.iloc[i1].to_numpy(dtype = object)
X_test = Z_df.iloc[i2].to_numpy(dtype = object)

In [25]:
y_train =df_data['IC50'].iloc[i1].to_numpy(dtype = object)
y_test = df_data['IC50'].iloc[i2].to_numpy(dtype = object)

In [ ]:
from sklearn.model_selection import train_test_split
y = df_final["IC50"]
X_train, X_test, y_train, y_test = train_test_split(foo2, y, test_size=0.2)

In [52]:
test_list = pd.isnull(y_train)
res = [i for i, val in enumerate(test_list) if val] 
y_train = np.delete(y_train,res,0)
X_train = np.delete(X_train,res,0)

In [35]:
X_train

array([[30.0, 8.0, 0.0, ..., 0.0, 0.0, 0.0],
       [30.0, 7.5, 0.0, ..., 0.0, 0.0, 0.0],
       [25.0, 6.7, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [23.0, 7.5, 0.0, ..., 0.0, 0.0, 0.0],
       [25.0, 7.5, 0.0, ..., 0.0, 0.0, 0.0],
       [37.0, 7.4, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(X_test)

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
tree_model = DecisionTreeRegressor()

In [54]:
tree_model.fit(train_scaled, y_train)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import average_precision_score
tree_mse = mean_squared_error(y_train, tree_model.predict(train_scaled))
tree_mae = mean_absolute_error(y_train, tree_model.predict(train_scaled))
from math import sqrt
print("Decision Tree training mse = ",tree_mse," & mae = ",tree_mae," & rmse = ", sqrt(tree_mse))
tree_test_mse = mean_squared_error(y_test, tree_model.predict(test_scaled))
tree_test_mae = mean_absolute_error(y_test, tree_model.predict(test_scaled))
print("Decision Tree test mse = ",tree_test_mse," & mae = ",tree_test_mae," & rmse = ", sqrt(tree_test_mse))
a = pearsonr(y_train, tree_model.predict(train_scaled))
b = pearsonr(y_test, tree_model.predict(test_scaled))
print("Decision Tree train r = ",a)
print("Decision Tree test r = ",b)

Decision Tree training mse =  47634.32296853293  & mae =  7.910555190057018  & rmse =  218.25288765222083
Decision Tree test mse =  11179969184036.688  & mae =  143895.53864962212  & rmse =  3343646.0913255587
Decision Tree train r =  (0.9999999973772045, 0.0)
Decision Tree test r =  (0.6210591543274003, 0.0)


In [59]:
importances = pd.DataFrame({'importance':np.round(tree_model.feature_importances_,3)})
out = importances.sort_values('importance',ascending=False)
importances["Type"] = typelist
type_importance = importances.groupby(by=['Type']).sum()
type_importance.sort_values('importance',ascending=False)

,importance
Type,
smiles2daylight,0.495
protein2emb_encoder,0.424
drug2emb_encoder,0.073
CalculateConjointTriad,0.002
organism,0.002
calcPubChemFingerAll,0.001
smiles2morgan,0.001
Temp,0.000
pH,0.000


In [28]:
X.head()

,Kd,Ki,EC50,Temp,pH,smiles2morgan,drug2emb_encoder,CalculateConjointTriad,protein2emb_encoder,var_Abelson murine leukemia virus,...,var_Influenza B virus,var_Influenza B virus (B/Memphis/3/93),var_Influenza B virus (B/Victoria/517/2005),var_Influenza B virus (strain B/Lee/1940),var_Influenza B virus (strain B/Memphis/3/1989),var_Klebsiella pneumoniae,var_Mus musculus,var_Oryctolagus cuniculus,var_Pseudomonas aeruginosa,var_Rattus norvegicus
0,872669.450093,0.24,362878.629538,37.0,5.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([515, 343, 982, 52, 93, 210, 614, 1244, 690, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,...,0,0,0,0,0,0,0,0,0,0
1,872669.450093,0.25,362878.629538,37.0,5.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([1138, 186, 144, 265, 199, 188, 381, 1734, 13...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,...,0,0,0,0,0,0,0,0,0,0
2,872669.450093,0.41,362878.629538,37.0,5.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([1138, 186, 144, 265, 261, 158, 322, 65, 188,...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,...,0,0,0,0,0,0,0,0,0,0
3,872669.450093,0.80,362878.629538,37.0,5.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([700, 409, 1769, 833, 144, 265, 261, 158, 258...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,...,0,0,0,0,0,0,0,0,0,0
4,872669.450093,0.99,362878.629538,37.0,5.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([700, 223, 1769, 833, 144, 265, 261, 158, 258...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
importances

,importance
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
5663,0.0
5664,0.0
5665,0.0
5666,0.0


In [57]:
out.head(20)

,importance
4472,0.424
4021,0.377
3715,0.115
1035,0.073
2633,0.003
4108,0.002
5633,0.002
1430,0.001
342,0.001
3777,0.000


In [61]:
rf_model = RandomForestRegressor(n_estimators=10)
rf_model.fit(train_scaled, y_train)

RandomForestRegressor(n_estimators=10)

In [65]:
a = pearsonr(y_train, loaded_model.predict(train_scaled))
b = pearsonr(y_test, loaded_model.predict(test_scaled))
print("Random Forest train r = ",a)
print("Random Forest test r = ",b)

Random Forest train r =  (0.9590994255803869, 0.0)
Random Forest test r =  (0.7822213949408889, 0.0)


In [63]:
filename = 'rf_model.sav'
pickle.dump(rf_model, open(filename, 'wb'))

In [64]:
loaded_model = pickle.load(open(filename, 'rb'))

In [66]:
importances = pd.DataFrame({'importance':np.round(rf_model.feature_importances_,3)})
out = importances.sort_values('importance',ascending=False)
importances["Type"] = typelist
type_importance = importances.groupby(by=['Type']).sum()
type_importance.sort_values('importance',ascending=False)

,importance
Type,
protein2emb_encoder,0.493
smiles2daylight,0.341
drug2emb_encoder,0.128
calcPubChemFingerAll,0.024
smiles2morgan,0.009
organism,0.001
CalculateConjointTriad,0.000
Temp,0.000
pH,0.000


In [ ]:
rf_mse = mean_squared_error(y_train, rf_model.predict(train_scaled))
rf_mae = mean_absolute_error(y_train, rf_model.predict(train_scaled))

print("Random Forest training mse = ",rf_mse," & mae = ",rf_mae," & rmse = ", sqrt(rf_mse))
rf_test_mse = mean_squared_error(y_test, rf_model.predict(test_scaled))
rf_test_mae = mean_absolute_error(y_test, rf_model.predict(test_scaled))
print("Random Forest test mse = ",rf_test_mse," & mae = ",rf_test_mae," & rmse = ", sqrt(rf_test_mse))

In [ ]:
from boruta import BorutaPy

###initialize Boruta
forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5
)
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 10 # number of trials to perform
)
### fit Boruta (it accepts np.array, not pd.DataFrame)
boruta.fit(np.array(X_train), np.array(y_train))
### print results
green_area = X.columns[boruta.support_].to_list()
blue_area = X.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

In [26]:
# dti split

split_method = 'cold_drug'
random_seed = 1
frac = [0.7, 0.2, 0.1]

print('splitting dataset...')

#TODO: what is HTS

if split_method == 'random': 
    train, val, test = create_fold(df_data, random_seed, frac)
elif split_method == 'cold_drug':
    train, val, test = create_fold_setting_cold_drug(df_data, random_seed, frac)
elif split_method == 'HTS':
    train, val, test = create_fold_setting_cold_drug(df_data, random_seed, frac)
    val = pd.concat([val[val.Label == 1].drop_duplicates(subset = 'SMILES'), val[val.Label == 0]])
    test = pd.concat([test[test.Label == 1].drop_duplicates(subset = 'SMILES'), test[test.Label == 0]])        
elif split_method == 'cold_protein':
    train, val, test = create_fold_setting_cold_protein(df_data, random_seed, frac)
elif split_method == 'repurposing_VS':
    train = df_data
    val = df_data
    test = df_data
elif split_method == 'no_split':
    print('do not do train/test split on the data for already splitted data')
else:
    raise AttributeError("Please select one of the three split method: random, cold_drug, cold_target!")
    
print('Done.')

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)
    

splitting dataset...


KeyError: 'SMILES'

In [ ]:
train

In [ ]:

result_folder = "./result/"
input_dim_drug = 1024
input_dim_protein = 8420
hidden_dim_drug = 256
hidden_dim_protein = 256
cls_hidden_dims = [1024, 1024, 512]
mlp_hidden_dims_drug = [1024, 256, 64]
mlp_hidden_dims_target = [1024, 256, 64]
batch_size = 256
train_epoch = 10
test_every_X_epoch = 20
LR = 1e-4
decay = 0
transformer_emb_size_drug = 128
transformer_intermediate_size_drug = 512
transformer_num_attention_heads_drug = 8
transformer_n_layer_drug = 8
transformer_emb_size_target = 64
transformer_intermediate_size_target = 256
transformer_num_attention_heads_target = 4
transformer_n_layer_target = 2
transformer_dropout_rate = 0.1
transformer_attention_probs_dropout = 0.1
transformer_hidden_dropout_rate = 0.1
mpnn_hidden_size = 50
mpnn_depth = 3
cnn_drug_filters = [32,64,96]
cnn_drug_kernels = [4,6,8]
cnn_target_filters = [32,64,96]
cnn_target_kernels = [4,8,12]
rnn_Use_GRU_LSTM_drug = 'GRU'
rnn_drug_hid_dim = 64
rnn_drug_n_layers = 2
rnn_drug_bidirectional = True
rnn_Use_GRU_LSTM_target = 'GRU'
rnn_target_hid_dim = 64
rnn_target_n_layers = 2
rnn_target_bidirectional = True
num_workers = 0 

base_config = {'input_dim_drug': input_dim_drug,
                'input_dim_protein': input_dim_protein,
                'hidden_dim_drug': hidden_dim_drug, # hidden dim of drug
                'hidden_dim_protein': hidden_dim_protein, # hidden dim of protein
                'cls_hidden_dims' : cls_hidden_dims, # decoder classifier dim 1
                'batch_size': batch_size,
                'train_epoch': train_epoch,
                'test_every_X_epoch': test_every_X_epoch, 
                'LR': LR,
                'result_folder': result_folder,
                'binary': False,
                'num_workers' : num_workers,
                'result_folder' : "./result/",
                'input_dim_drug' : 1024,
                'input_dim_protein': 8420,
                'hidden_dim_drug': 256,
                'hidden_dim_protein': 256,
                'cls_hidden_dims': [1024, 1024, 512],
                'mlp_hidden_dims_drug': [1024, 256, 64],
                'mlp_hidden_dims_target': [1024, 256, 64],
                'batch_size': 256,
                'train_epoch': 10,
                'test_every_X_epoch': 20,
                'LR': 1e-4,
                'decay': 0,
                'transformer_emb_size_drug': 128,
                'transformer_intermediate_size_drug': 512,
                'transformer_num_attention_heads_drug': 8,
                'transformer_n_layer_drug': 8,
                'transformer_emb_size_target': 64,
                'transformer_intermediate_size_target': 256,
                'transformer_num_attention_heads_target': 4,
                'transformer_n_layer_target': 2,
                'transformer_dropout_rate': 0.1,
                'transformer_attention_probs_dropout': 0.1,
                'transformer_hidden_dropout_rate': 0.1,
                'mpnn_hidden_size': 50,
                'mpnn_depth': 3,
                'cnn_drug_filters': [32,64,96],
                'cnn_drug_kernels': [4,6,8],
                'cnn_target_filters': [32,64,96],
                'cnn_target_kernels': [4,8,12],
                'rnn_Use_GRU_LSTM_drug': 'GRU',
                'rnn_drug_hid_dim': 64,
                'rnn_drug_n_layers': 2,
                'rnn_drug_bidirectional' : True,
                'rnn_Use_GRU_LSTM_target' : 'GRU',
                'rnn_target_hid_dim' : 64,
                'rnn_target_n_layers' : 2,
                'rnn_target_bidirectional' : True,
                'num_workers' : 0 
}
base_config['result_folder']

In [ ]:
if not os.path.exists(base_config['result_folder']):
    os.makedirs(base_config['result_folder'])

base_config['mlp_hidden_dims_drug'] = mlp_hidden_dims_drug # MLP classifier dim 1				
base_config['input_dim_drug'] = 881 #could be 2048 or 200 or 2586
base_config['cnn_drug_filters'] = cnn_drug_filters
base_config['cnn_drug_kernels'] = cnn_drug_kernels
base_config['rnn_Use_GRU_LSTM_drug'] = rnn_Use_GRU_LSTM_drug
base_config['rnn_drug_hid_dim'] = rnn_drug_hid_dim
base_config['rnn_drug_n_layers'] = rnn_drug_n_layers
base_config['rnn_drug_bidirectional'] = rnn_drug_bidirectional 
base_config['transformer_emb_size_drug'] = transformer_emb_size_drug
base_config['transformer_num_attention_heads_drug'] = transformer_num_attention_heads_drug
base_config['transformer_intermediate_size_drug'] = transformer_intermediate_size_drug
base_config['transformer_n_layer_drug'] = transformer_n_layer_drug
base_config['transformer_dropout_rate'] = transformer_dropout_rate
base_config['transformer_attention_probs_dropout'] = transformer_attention_probs_dropout
base_config['transformer_hidden_dropout_rate'] = transformer_hidden_dropout_rate
base_config['hidden_dim_drug'] = transformer_emb_size_drug #could also be hidden_dim_drug
base_config['batch_size'] = batch_size 
base_config['mpnn_hidden_size'] = mpnn_hidden_size
base_config['mpnn_depth'] = mpnn_depth

base_config['mlp_hidden_dims_target'] = mlp_hidden_dims_target # MLP classifier dim 1				
base_config['input_dim_protein'] = 30 #could be 343 or 100 or 4114
base_config['cnn_target_filters'] = cnn_target_filters
base_config['cnn_target_kernels'] = cnn_target_kernels
base_config['rnn_Use_GRU_LSTM_target'] = rnn_Use_GRU_LSTM_target
base_config['rnn_target_hid_dim'] = rnn_target_hid_dim
base_config['rnn_target_n_layers'] = rnn_target_n_layers
base_config['rnn_target_bidirectional'] = rnn_target_bidirectional 
base_config['cnn_target_filters'] = cnn_target_filters
base_config['cnn_target_kernels'] = cnn_target_kernels
base_config['transformer_emb_size_target'] = transformer_emb_size_target
base_config['transformer_num_attention_heads_target'] = transformer_num_attention_heads_target
base_config['transformer_intermediate_size_target'] = transformer_intermediate_size_target
base_config['transformer_n_layer_target'] = transformer_n_layer_target	
base_config['transformer_dropout_rate'] = transformer_dropout_rate
base_config['transformer_attention_probs_dropout'] = transformer_attention_probs_dropout
base_config['transformer_hidden_dropout_rate'] = transformer_hidden_dropout_rate
base_config['hidden_dim_protein'] = transformer_emb_size_target

config = base_config

In [ ]:
from collections import namedtuple
model_drug_tuple = namedtuple("model_drug_tuple", "MLP CNN CNN_RNN transformer MPNN")
model_protein_tuple = namedtuple("model_protein_tuple", "MLP CNN CNN_RNN transformer")

model_drug_MLP = MLP(config['input_dim_drug'], config['hidden_dim_drug'], config['mlp_hidden_dims_drug'])
model_drug_CNN = CNN('drug', **config)
model_drug_CNN_RNN = CNN_RNN('drug', **config)
model_drug_transformer = transformer('drug', **config)
model_drug_MPNN = MPNN(config['hidden_dim_drug'], config['mpnn_depth'])

model_drug = model_drug_tuple(model_drug_MLP, model_drug_CNN, model_drug_CNN_RNN, model_drug_transformer, model_drug_MPNN)

model_protein_MLP = MLP(config['input_dim_protein'], config['hidden_dim_protein'], config['mlp_hidden_dims_target'])
model_protein_CNN = CNN('protein', **config)
model_protein_CNN_RNN = CNN_RNN('protein', **config)
model_protein_transformer = transformer('protein', **config)

model_protein = model_protein_tuple(model_protein_MLP, model_protein_CNN, model_protein_CNN_RNN, model_protein_transformer)

model_feature_tuple = namedtuple("model_feature_tuple","model_drug model_protein model_df")
model_features = model_feature_tuple(model_drug, model_protein, train)

In [ ]:
class Classifier_o(nn.Sequential):
    def __init__(self, model_struct, **config):
        super(Classifier_o, self).__init__()
        self.input_dim_drug = config['hidden_dim_drug']
        self.input_dim_protein = config['hidden_dim_protein']

        self.model_struct = model_struct

        self.dropout = nn.Dropout(0.1)

        self.hidden_dims = config['cls_hidden_dims']
        layer_size = len(self.hidden_dims) + 1
        dims = [self.input_dim_drug + self.input_dim_protein] + self.hidden_dims + [1]

        self.predictor = nn.ModuleList([nn.Linear(dims[i], dims[i+1]) for i in range(layer_size)])

    def forward(self, v_D, v_P):
        # each encoding
        v_D = self.model_struct.model_drug(v_D)
        v_P = self.model_struct.model_protein(v_P)
        # concatenate and classify
        v_f = torch.cat((v_D, v_P), 1)
        for i, l in enumerate(self.predictor):
            if i==(len(self.predictor)-1):
                v_f = l(v_f)
            else:
                v_f = F.relu(self.dropout(l(v_f)))
        return v_f

In [ ]:
model = Classifier_o(model_features, **config)

In [ ]:
input_dim_drug = config['hidden_dim_drug']
input_dim_protein = config['hidden_dim_protein']
model_drug = model_drug
model_protein = model_protein
dropout = nn.Dropout(0.1)
hidden_dims = config['cls_hidden_dims']
layer_size = len(hidden_dims) + 1
dims = [input_dim_drug + input_dim_protein] + hidden_dims + [1]
predictor = nn.ModuleList([nn.Linear(dims[i], dims[i+1]) for i in range(layer_size)])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
result_folder = config['result_folder']
       
binary = False

In [ ]:
lr = config['LR']
decay = config['decay']
BATCH_SIZE = config['batch_size']
train_epoch = config['train_epoch']
loss_history = []
verbose = True

model = model.to(device)

# support multiple GPUs
if torch.cuda.device_count() > 1:
    if verbose:
        print("Let's use " + str(torch.cuda.device_count()) + " GPUs!")
    model = nn.DataParallel(model, dim = 0)
elif torch.cuda.device_count() == 1:
    if verbose:
        print("Let's use " + str(torch.cuda.device_count()) + " GPU!")
else:
    if verbose:
        print("Let's use CPU/s!")
# Future TODO: support multiple optimizers with parameters
opt = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = decay)
if verbose:
    print('--- Data Preparation ---')

params = {'batch_size': BATCH_SIZE,
        'shuffle': True,
        'num_workers': config['num_workers'],
        'drop_last': False}

params['collate_fn'] = DTI.mpnn_collate_func

In [ ]:
class data_process_loader_o(data.Dataset):

    def __init__(self, list_IDs, df, **config):
        'Initialization'
        self.list_IDs = list_IDs
        self.df = df
        self.config = config

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        index = self.list_IDs[index]
        v = self.df.iloc[index]
#        v_d = self.df.iloc[index]['drug_encoding']        
#        v_d = drug_2_embed(v_d)
#        v_p = self.df.iloc[index]['target_encoding']
#        v_p = protein_2_embed(v_p)
        #y = self.labels[index]
        return v

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc_protein = OneHotEncoder().fit(np.array(amino_char).reshape(-1, 1))
enc_drug = OneHotEncoder().fit(np.array(smiles_char).reshape(-1, 1))

def protein_2_embed(x):
	return enc_protein.transform(np.array(x).reshape(-1,1)).toarray().T
def drug_2_embed(x):
	return enc_drug.transform(np.array(x).reshape(-1,1)).toarray().T    

In [ ]:
from torch.utils.data.dataset import Dataset

class MyCustomDataset(Dataset):
    def __init__(self, ...):
        # stuff
        
    def __getitem__(self, index):
        # stuff
        return (img, label)

    def __len__(self):
        return count

In [ ]:
training_generator = torch.utils.data.DataLoader(train, **params)
validation_generator = torch.utils.data.DataLoader(val, **params)

In [ ]:
if test is not None:
    info = data_process_loader_o(test.index.values,  test, **config)
    params_test = {'batch_size': BATCH_SIZE,
            'shuffle': False,
            'num_workers': config['num_workers'],
            'drop_last': False,
            'sampler':SequentialSampler(info)}
    params_test['collate_fn'] = DTI.mpnn_collate_func
    testing_generator = data.DataLoader(data_process_loader_o(test.index.values, test, **config), **params_test)

# early stopping
if binary:
    max_auc = 0
else:
    max_MSE = 10000
model_max = copy.deepcopy(model)

valid_metric_record = []
valid_metric_header = ["# epoch"] 
if binary:
    valid_metric_header.extend(["AUROC", "AUPRC", "F1"])
else:
    valid_metric_header.extend(["MSE", "Pearson Correlation", "with p-value", "Concordance Index"])
table = PrettyTable(valid_metric_header)
float2str = lambda x:'%0.4f'%x
if verbose:
    print('--- Go for Training ---')
t_start = time() 
for epo in range(train_epoch):
    for v in enumerate(training_generator):
        if self.target_encoding == 'Transformer':
            v_p = v_p
        else:
            v_p = v_p.float().to(self.device) 
        if self.drug_encoding == "MPNN" or self.drug_encoding == 'Transformer':
            v_d = v_d
        else:
            v_d = v_d.float().to(self.device)                
            #score = self.model(v_d, v_p.float().to(self.device))

        score = self.model(v_d, v_p)
        label = Variable(torch.from_numpy(np.array(label)).float()).to(self.device)

        if self.binary:
            loss_fct = torch.nn.BCELoss()
            m = torch.nn.Sigmoid()
            n = torch.squeeze(m(score), 1)
            loss = loss_fct(n, label)
        else:
            loss_fct = torch.nn.MSELoss()
            n = torch.squeeze(score, 1)
            loss = loss_fct(n, label)
        loss_history.append(loss.item())

        opt.zero_grad()
        loss.backward()
        opt.step()

        if verbose:
            if (i % 100 == 0):
                t_now = time()
                print('Training at Epoch ' + str(epo + 1) + ' iteration ' + str(i) + \
                    ' with loss ' + str(loss.cpu().detach().numpy())[:7] +\
                    ". Total time " + str(int(t_now - t_start)/3600)[:7] + " hours") 
                ### record total run time

    ##### validate, select the best model up to now 
    with torch.set_grad_enabled(False):
        if self.binary:  
            ## binary: ROC-AUC, PR-AUC, F1, cross-entropy loss
            auc, auprc, f1, loss, logits = self.test_(validation_generator, self.model)
            lst = ["epoch " + str(epo)] + list(map(float2str,[auc, auprc, f1]))
            valid_metric_record.append(lst)
            if auc > max_auc:
                model_max = copy.deepcopy(self.model)
                max_auc = auc   
            if verbose:
                print('Validation at Epoch '+ str(epo + 1) + ' , AUROC: ' + str(auc)[:7] + \
                  ' , AUPRC: ' + str(auprc)[:7] + ' , F1: '+str(f1)[:7] + ' , Cross-entropy Loss: ' + \
                  str(loss)[:7])
        else:  
            ### regression: MSE, Pearson Correlation, with p-value, Concordance Index  
            mse, r2, p_val, CI, logits = self.test_(validation_generator, self.model)
            lst = ["epoch " + str(epo)] + list(map(float2str,[mse, r2, p_val, CI]))
            valid_metric_record.append(lst)
            if mse < max_MSE:
                model_max = copy.deepcopy(self.model)
                max_MSE = mse
            if verbose:
                print('Validation at Epoch '+ str(epo + 1) + ' , MSE: ' + str(mse)[:7] + ' , Pearson Correlation: '\
                 + str(r2)[:7] + ' with p-value: ' + str(p_val)[:7] +' , Concordance Index: '+str(CI)[:7])
    table.add_row(lst)


# load early stopped model
self.model = model_max

#### after training 
prettytable_file = os.path.join(self.result_folder, "valid_markdowntable.txt")
with open(prettytable_file, 'w') as fp:
    fp.write(table.get_string())

if test is not None:
    if verbose:
        print('--- Go for Testing ---')
    if self.binary:
        auc, auprc, f1, loss, logits = self.test_(testing_generator, model_max, test = True)
        test_table = PrettyTable(["AUROC", "AUPRC", "F1"])
        test_table.add_row(list(map(float2str, [auc, auprc, f1])))
        if verbose:
            print('Validation at Epoch '+ str(epo + 1) + ' , AUROC: ' + str(auc)[:7] + \
              ' , AUPRC: ' + str(auprc)[:7] + ' , F1: '+str(f1)[:7] + ' , Cross-entropy Loss: ' + \
              str(loss)[:7])				
    else:
        mse, r2, p_val, CI, logits = self.test_(testing_generator, model_max)
        test_table = PrettyTable(["MSE", "Pearson Correlation", "with p-value", "Concordance Index"])
        test_table.add_row(list(map(float2str, [mse, r2, p_val, CI])))
        if verbose:
            print('Testing MSE: ' + str(mse) + ' , Pearson Correlation: ' + str(r2) 
              + ' with p-value: ' + str(p_val) +' , Concordance Index: '+str(CI))
    np.save(os.path.join(self.result_folder, str(self.drug_encoding) + '_' + str(self.target_encoding) 
             + '_logits.npy'), np.array(logits))                

    ######### learning record ###########

    ### 1. test results
    prettytable_file = os.path.join(self.result_folder, "test_markdowntable.txt")
    with open(prettytable_file, 'w') as fp:
        fp.write(test_table.get_string())

### 2. learning curve 
fontsize = 16
iter_num = list(range(1,len(loss_history)+1))
plt.figure(3)
plt.plot(iter_num, loss_history, "bo-")
plt.xlabel("iteration", fontsize = fontsize)
plt.ylabel("loss value", fontsize = fontsize)
pkl_file = os.path.join(self.result_folder, "loss_curve_iter.pkl")
with open(pkl_file, 'wb') as pck:
    pickle.dump(loss_history, pck)

fig_file = os.path.join(self.result_folder, "loss_curve.png")
plt.savefig(fig_file)
if verbose:
    print('--- Training Finished ---')